In [ ]:
 !pip install -q google-generativeai pandas streamlit gspread oauth2client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 405.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# After drive.mount() is successful
service_account_path = '/content/drive/MyDrive/GCP Projects/service-account-key.json'
import gspread
gc = gspread.service_account(filename=service_account_path)
print("gspread authenticated via Service Account JSON from Drive.")

gspread authenticated via Service Account JSON from Drive.


In [ ]:
# Cell 3: Read Google Sheet Structure and Data (after API key setup)

import gspread
from google.colab import userdata
import json
import os # Needed for os.getenv if you use that for service account key path

# --- Authentication (choose the one you set up) ---
# Option A: From file on Drive (if drive is mounted AND path is correct)
service_account_path = '/content/drive/MyDrive/GCP Projects/service-account-key.json' # REPLACE with your actual path
gc = gspread.service_account(filename=service_account_path)

# Option B: From Colab Secret (Recommended and more robust for Colab)
# sa_json_str = userdata.get("GSHEETS_SERVICE_ACCOUNT_JSON")
# if not sa_json_str:
#     raise ValueError("GSHEETS_SERVICE_ACCOUNT_JSON secret not found. Please add it to Colab Secrets.")
# sa_creds = json.loads(sa_json_str) # Load JSON string into dictionary
# gc = gspread.service_account_from_dict(sa_creds)

# print("gspread authenticated successfully.")

# --- Specify your Spreadsheet ---
# You can use the full URL or just the Spreadsheet ID
# spreadsheet_url = "https://docs.google.com/spreadsheets/d/YOUR_SPREADSHEET_ID_HERE/edit#gid=0"
# spreadsheet_id = "YOUR_SPREADSHEET_ID_HERE" # Get this from the URL: https://docs.google.com/spreadsheets/d/THIS_IS_THE_ID/edit#gid=0

# For simplicity, let's use the name of the spreadsheet you created in Step 5
spreadsheet_name = "Project Trial -1" # Make sure this is the exact name!

try:
    spreadsheet = gc.open(spreadsheet_name)
    print(f"Successfully opened spreadsheet: {spreadsheet_name}")
except gspread.exceptions.SpreadsheetNotFound:
    print(f"Error: Spreadsheet '{spreadsheet_name}' not found. Please check the name and ensure your service account has Editor access.")
    raise # Re-raise the exception to stop execution

# --- Get Dashboard Template Layout ---
dashboard_template_sheet_name = "Template"
try:
    dashboard_template_sheet = spreadsheet.worksheet(dashboard_template_sheet_name)
    # Read all values, including empty cells, to preserve layout structure
    dashboard_layout_values = dashboard_template_sheet.get_all_values()
    print(f"Successfully read layout from '{dashboard_template_sheet_name}' tab.")
    # Print a snippet to verify
    print("Dashboard Layout Sample (first 3 rows):")
    for row in dashboard_layout_values[:3]:
        print(row)
except gspread.exceptions.WorksheetNotFound:
    print(f"Error: Tab '{dashboard_template_sheet_name}' not found. Please check the tab name.")
    raise

# --- Get Raw Data Headers ---
raw_data_sheet_name = "Raw_data"
try:
    raw_data_sheet = spreadsheet.worksheet(raw_data_sheet_name)
    raw_data_headers = raw_data_sheet.row_values(1) # Assumes headers are in the first row
    print(f"Successfully read headers from '{raw_data_sheet_name}' tab.")
    print("Raw Data Headers:", raw_data_headers)
except gspread.exceptions.WorksheetNotFound:
    print(f"Error: Tab '{raw_data_sheet_name}' not found. Please check the tab name.")
    raise

# Store these for later steps
# You'll use dashboard_layout_values and raw_data_headers in LLM prompting!

Successfully opened spreadsheet: Project Trial -1
Successfully read layout from 'Template' tab.
Dashboard Layout Sample (first 3 rows):
['Date', '6/13/2025', '6/12/2025', '6/11/2025']
['BEL', '', '', '']
['Open Price', '', '', '']
Successfully read headers from 'Raw_data' tab.
Raw Data Headers: ['Symbol  ', 'Series  ', 'Date  ', 'Prev Close  ', 'Open Price  ', 'High Price  ', 'Low Price  ', 'Last Price  ', 'Close Price  ', 'Average Price ', 'Total Traded Quantity  ', 'Turnover ₹  ', 'No. of Trades  ', 'Deliverable Qty  ', '% Dly Qt to Traded Qty  ']


In [ ]:
# # --- DEBUGGING CELL ---
# # Run this cell right before your main script to check the setup.

# print("--- DEBUGGING INFO ---")
# try:
#     # 1. Check if the worksheet object is correct
#     print(f"Connected to worksheet titled: '{dashboard_template_sheet.title}'")
#     print(f"Spreadsheet URL: {spreadsheet.url}")

#     # 2. Check what's ACTUALLY in your configured start cells
#     print("--------------------------------------------------")
#     print(f"Configuration says METRICS start at: {METRICS_START_CELL}")
#     metric_value = dashboard_template_sheet.acell(METRICS_START_CELL).value
#     print(f"Value found in cell {METRICS_START_CELL}: '{metric_value}'")
#     if not metric_value:
#         print("  -> WARNING: This cell is empty! The script will not find any metrics.")

#     print(f"\nConfiguration says DATES start at: {DATES_START_CELL}")
#     date_value = dashboard_template_sheet.acell(DATES_START_CELL).value
#     print(f"Value found in cell {DATES_START_CELL}: '{date_value}'")
#     if not date_value:
#         print("  -> WARNING: This cell is empty! The script will not find any dates.")

#     # 3. Check the freshness of the data snapshot
#     print("--------------------------------------------------")
#     print(f"The data snapshot 'dashboard_layout_values' has {len(dashboard_layout_values)} rows.")
#     print("If you recently changed your sheet, make sure you re-ran the cell that defines this variable.")

# except NameError as e:
#     print(f"\nERROR: A required variable is not defined: {e}")
#     print("Please make sure you have run the cells that define 'spreadsheet' and 'dashboard_template_sheet'.")
# except Exception as e:
#     print(f"\nAn unexpected error occurred during debugging: {e}")

# print("\n--- END DEBUGGING INFO ---")

--- DEBUGGING INFO ---
Connected to worksheet titled: 'Template'
Spreadsheet URL: https://docs.google.com/spreadsheets/d/1aMVKx7jkh0xRaxEn6HbkKWPlP8Cyen6zviXhs1dfMkA
--------------------------------------------------
Configuration says METRICS start at: A3
Value found in cell A3: 'Open Price'

Configuration says DATES start at: B1
Value found in cell B1: '6/13/2025'
--------------------------------------------------
The data snapshot 'dashboard_layout_values' has 7 rows.
If you recently changed your sheet, make sure you re-ran the cell that defines this variable.

--- END DEBUGGING INFO ---


In [ ]:
# Cell 4: Final, Complete Script with Formula Generation Engine

import google.generativeai as genai
from google.colab import userdata
import os
import re
import gspread
import time

# --- Helper functions (No changes needed) ---
def get_column_letter(col_num):
    string = ""
    while col_num > 0:
        col_num, remainder = divmod(col_num - 1, 26)
        string = chr(65 + remainder) + string
    return string

# --- API Configuration (No changes needed) ---
api_key = userdata.get("GOOGLE_API_KEY")
if not api_key: raise ValueError("GOOGLE_API_KEY not set.")
genai.configure(api_key=api_key)
gemini_model = genai.GenerativeModel('models/gemini-1.5-flash-latest')
print("Gemini model configured.")

# --- Automatic Data Refresh ---
# Assumes the 'spreadsheet' object is defined in a previous cell.
try:
    dashboard_template_sheet = spreadsheet.worksheet("Template") # Use your exact tab name
    dashboard_layout_values = dashboard_template_sheet.get_all_values()
    print(f"Successfully fetched a fresh data snapshot from '{dashboard_template_sheet.title}' sheet.")
except gspread.exceptions.WorksheetNotFound:
    print("ERROR: The worksheet named 'Template' was not found. Please check the name.")
    # Stop execution if the sheet is not found
    raise
except Exception as e:
    print(f"An error occurred while fetching the sheet: {e}")
    raise

# --- CONFIGURATION BLOCK ---
METRICS_START_CELL = 'A3'
DATES_START_CELL = 'B1'
MASTER_SYMBOL_CELL = 'A2'
RAW_DATA_SHEET_NAME = 'Raw_data'
print(f"Configuration loaded: Metrics from {METRICS_START_CELL}, Dates from {DATES_START_CELL}")

# --- Robust Logic to Read the Template Structure ---
formulas_to_insert = []
print("\nReading template structure based on your configuration...")
try:
    metrics_start_row, metrics_start_col = gspread.utils.a1_to_rowcol(METRICS_START_CELL)
    dates_start_row, dates_start_col = gspread.utils.a1_to_rowcol(DATES_START_CELL)

    metric_labels_with_rownum = []
    all_metrics_in_col = dashboard_template_sheet.col_values(metrics_start_col)
    for r_idx in range(metrics_start_row - 1, len(all_metrics_in_col)):
        metric_label = all_metrics_in_col[r_idx].strip()
        if metric_label:
            metric_labels_with_rownum.append({'label': metric_label, 'row_num': r_idx + 1})
        else:
            break
    print(f"Found {len(metric_labels_with_rownum)} metrics.")

    date_labels_with_colnum = []
    all_dates_in_row = dashboard_template_sheet.row_values(dates_start_row)
    for c_idx in range(dates_start_col - 1, len(all_dates_in_row)):
        date_label = all_dates_in_row[c_idx].strip()
        if date_label:
            date_labels_with_colnum.append({'label': date_label, 'col_num': c_idx + 1})
        else:
            break
    print(f"Found {len(date_labels_with_colnum)} dates.")

    for metric_info in metric_labels_with_rownum:
        for date_info in date_labels_with_colnum:
            target_row_index = metric_info['row_num'] - 1
            target_col_index = date_info['col_num'] - 1
            cell_content = ''
            if target_row_index < len(dashboard_layout_values) and target_col_index < len(dashboard_layout_values[target_row_index]):
                 cell_content = dashboard_layout_values[target_row_index][target_col_index].strip()
            if not cell_content:
                formulas_to_insert.append({
                    'cell_a1': f"{get_column_letter(date_info['col_num'])}{metric_info['row_num']}",
                    'row_label': metric_info['label'],
                    'col_label': date_info['label'],
                    'row_num': metric_info['row_num'],
                    'col_num': date_info['col_num'],
                    'header_row_num': dates_start_row
                })
except Exception as e:
    print(f"\nAn unexpected error occurred while reading the template: {e}")

print(f"\nIdentified {len(formulas_to_insert)} cells to fill with formulas.")


# ==============================================================================
# === THE FORMULA GENERATION ENGINE (This was the missing part) ===
# ==============================================================================
generated_formulas = []
if formulas_to_insert:
    # This check is now true, so this block will run
    raw_data_headers = spreadsheet.worksheet(RAW_DATA_SHEET_NAME).row_values(1)
    raw_data_header_to_col_letter = {h.strip(): get_column_letter(i + 1) for i, h in enumerate(raw_data_headers)}

    prompt_instructions = f"""
You are a world-class Google Sheets expert specializing in creating dynamic, efficient, and robust financial dashboards.
Your mission is to generate a single, draggable Google Sheets formula based on the rules and layout definitions below.

---
### **GOLDEN RULES (NON-NEGOTIABLE)**
---
**1. USE THE CORRECT SHEET NAME:**
   - All references to the raw data source MUST use the sheet name '{RAW_DATA_SHEET_NAME}'.
   - Example: `'{RAW_DATA_SHEET_NAME}'!A:A`.
   - NEVER use default names like 'Sheet1' or 'Sheet2'. This is a critical instruction.
**2. ALWAYS Use Aggregation Functions:**
   - You MUST use aggregation functions like `SUMIFS`, `AVERAGEIFS`, `MAXIFS`, `MINIFS`.
   - You MUST NOT use lookup functions like `VLOOKUP` or `INDEX(MATCH)`.
**3. Formulas MUST Be Draggable:**
   - You MUST use absolute `$` references correctly.
---
### **LAYOUT DEFINITIONS (THE "MAP")**
---
**1. '{RAW_DATA_SHEET_NAME}' Sheet Layout:**
   - Column A: 'Symbol'
   - Column C: 'Date'
   - Column E: 'Open Price'
   - Column F: 'High Price'
   - Column G: 'Low Price'
   - Column I: 'Close Price'
   - Column K: 'Total Traded Quantity'
   - Column L: 'Turnover ₹'
**2. 'Dashboard' Sheet Layout:**
   - **Master Symbol Filter:** Cell `{MASTER_SYMBOL_CELL}` is the master filter. All formulas MUST filter by the symbol in `${MASTER_SYMBOL_CELL}`.
   - **Metric Labels (Row Headers):** Start in `{METRICS_START_CELL}`. Reference with locked column (e.g., `$A3`).
   - **Date Headers (Column Headers):** Start in `{DATES_START_CELL}`. Reference with locked row (e.g., `B$1`).
---
### **THE PERFECT BLUEPRINT (EXAMPLE)**
---
**Task:** Create a formula for dashboard cell `B3`. The metric label in `$A3` is "Open Price". The date is in `B$1`.
**Perfect Formula for `B3`:**
`=IFERROR(AVERAGEIFS('{RAW_DATA_SHEET_NAME}'!$E:$E, '{RAW_DATA_SHEET_NAME}'!$C:$C, B$1, '{RAW_DATA_SHEET_NAME}'!$A:$A, $A$2), "")`

Now, use all of these rules and the provided blueprint to generate the formula for the specific task given below.
"""

    metrics_col_letter = get_column_letter(gspread.utils.a1_to_rowcol(METRICS_START_CELL)[1])

    for i, item in enumerate(formulas_to_insert):
        cell_a1 = item['cell_a1']
        row_label = item['row_label']

        dashboard_metric_cell_ref = f"${metrics_col_letter}{item['row_num']}"
        date_header_col_letter = get_column_letter(item['col_num'])
        dashboard_date_header_cell_ref = f"{date_header_col_letter}${item['header_row_num']}"

        prompt_task = f"""
        ---
        ### **CURRENT TASK**
        Generate a single, draggable Google Sheets formula for cell `{cell_a1}`.

        - The metric to calculate is specified by the row header: **"{row_label}"**.
        - The formula must be based on the date in the column header: `{dashboard_date_header_cell_ref}`.
        - Remember to always filter by the master symbol in cell `${MASTER_SYMBOL_CELL}`.

        Provide just the formula starting with =.

        **Google Sheets Formula:**
        """
        final_prompt = prompt_instructions
        print(f"  Generating formula for '{row_label}' at {cell_a1}...")

        try:
            response = gemini_model.generate_content(final_prompt + prompt_task)
            raw_text = response.text.strip()

            formula = raw_text
            match = re.search(r'```(?:excel|sheets|google sheets|gsheet)?\n(.*)```', raw_text, re.DOTALL)
            if match:
                formula = match.group(1).strip()

            formula = formula.replace("'Sheet1'!", f"'{RAW_DATA_SHEET_NAME}'!")
            formula = formula.replace("'Sheet2'!", f"'{RAW_DATA_SHEET_NAME}'!")

            formula = re.sub(r"^[=\s']+", "", formula).strip()
            cleaned_formula = "=" + formula

            generated_formulas.append({
                'cell_a1': cell_a1,
                'formula': cleaned_formula
            })
            print(f"    Generated: {cleaned_formula}")

        except Exception as e:
            print(f"    Error generating formula for {cell_a1}: {e}")

        time.sleep(6) # Pause between API calls
else:
    print("Could not identify any cells to fill. Check your configuration and template.")

print("\n--- SCRIPT FINISHED ---")

Gemini model configured.
Successfully fetched a fresh data snapshot from 'Template' sheet.
Configuration loaded: Metrics from A3, Dates from B1

Reading template structure based on your configuration...
Found 5 metrics.
Found 3 dates.

Identified 15 cells to fill with formulas.
  Generating formula for 'Open Price' at B3...
    Generated: =IFERROR(AVERAGEIFS('Raw_data'!$E:$E,'Raw_data'!$C:$C,B$1,'Raw_data'!$A:$A,$A$2),"")
  Generating formula for 'Open Price' at C3...
    Generated: =IFERROR(SUMIFS('Raw_data'!$E:$E,'Raw_data'!$C:$C,C$1,'Raw_data'!$A:$A,$A$2),"")
  Generating formula for 'Open Price' at D3...
    Generated: =IFERROR(AVERAGEIFS('Raw_data'!$E:$E,'Raw_data'!$C:$C,D$1,'Raw_data'!$A:$A,$A$2),"")
  Generating formula for 'High Price' at B4...
    Generated: =IFERROR(MAXIFS('Raw_data'!$F:$F,'Raw_data'!$C:$C,B$1,'Raw_data'!$A:$A,$A$2),"")
  Generating formula for 'High Price' at C4...
    Generated: =IFERROR(MAXIFS('Raw_data'!$F:$F,'Raw_data'!$C:$C,C$1,'Raw_data'!$A:$A,$A$2),""

In [ ]:
# Cell 5: Write Generated Formulas to Google Sheet (Correct Version)

print("\nAttempting to write generated formulas to Google Sheet...")

updates_for_sheet = []
for item in generated_formulas:
    sheet_name = dashboard_template_sheet.title
    updates_for_sheet.append({
        'range': f"'{sheet_name}'!{item['cell_a1']}",
        'values': [[item['formula']]]
    })

if updates_for_sheet:
    try:
        body = {
            'valueInputOption': 'USER_ENTERED', # This is the crucial line
            'data': updates_for_sheet
        }
        spreadsheet.values_batch_update(body=body)
        print(f"\n🎉 Formulas successfully written to '{sheet_name}'!")
    except Exception as e:
        print(f"\n❌ Error writing formulas to Google Sheet: {e}")
else:
    print("No formulas were generated to write.")


Attempting to write generated formulas to Google Sheet...

🎉 Formulas successfully written to 'Template'!
